In [6]:
#!/usr/bin/env python3

import unittest

from archemist.core.state.robot_op import RobotMaintenanceOp, RobotNavOp, RobotTaskOp
from archemist.core.state.robot import MobileRobot
from archemist.robots.kmriiwa_robot.state import KMRIIWARobot

from archemist.core.state.material import Liquid

from archemist.robots.kmriiwa_robot.handler import KmriiwaROSHandler
from archemist.core.state.batch import Batch
from archemist.core.state.lot import Lot
from archemist.core.util.enums import (
    StationState,
    OpOutcome,
    RobotState,
    MobileRobotMode,
)
from archemist.core.state.station_op_result import ProcessOpResult, MaterialOpResult
from archemist.core.persistence.models_proxy import ModelProxy, ListProxy, DictProxy
from archemist.stations.waters_lcms_station.handler import SimWatersLCMSStationHandler

from archemist.core.state.robot_op import RobotTaskOp, RobotWaitOp
from archemist.core.state.batch import Batch
from archemist.core.state.lot import Lot
from archemist.core.util.enums import OpOutcome, ProcessStatus
from archemist.core.util import Location


from datetime import datetime

import time

from mongoengine import connect

print("importing done")


class APCKukaRobotHandlerTest(unittest.TestCase):
    def setUp(self):
        self._db_name = "archemist_test"
        self._client = connect(
            db=self._db_name, host="mongodb://localhost:27017", alias="archemist_state"
        )

        self.robot_doc = {
            "type": "Mobile Robot",
            "id": 1,
            "handler": "KmriiwaROSHandler",
            "total_lot_capacity": 1,
            "onboard_capacity": 1,
        }

    def test_handler(self):
        robot = KMRIIWARobot.from_dict(self.robot_doc)
        self.assertIsNotNone(robot)
        self.assertEqual(robot.state, RobotState.INACTIVE)

        location_dict = {"coordinates": [18, 8], "descriptor": "Example Location"}

        # construct handler
        handler = KmriiwaROSHandler(robot)

        # initialise handler
        self.assertTrue(handler.initialise())

        target_loc = Location.from_dict(location_dict)

        # test Robot NavOp
        # construct op
        # t_op = RobotNavOp.from_args(
        #     name="test_nav",
        #     target_robot="Kuka 4",
        #     target_location=target_loc,
        #     params={},
        # )

        t_op = RobotTaskOp.from_args(
            name="test_task",
            target_robot="Kuka 4",
            target_location=target_loc,
            params={},
            lbr_program_name="test",
            lbr_program_params=[],
            fine_localization=True,
            task_type=1
        )

        robot.add_op(t_op)
        robot.update_assigned_op()
        self.assertEqual(robot.assigned_op, t_op)

        time.sleep(3)
        
        handler._kuka_cmd_seq = 10

        handler.execute_op()

        # wait for results
        handler.is_op_execution_complete()

        self.assertEqual(handler.is_op_execution_complete(), False)

        while True:
            if handler.is_op_execution_complete():
                print("got the completion msg, moving on")
                break
            else:
                print("didn't get the completion msg, hence trying")
            time.sleep(2)

        self.assertEqual(handler.is_op_execution_complete(), True)

        outcome = handler.get_op_result()

        self.assertEqual(outcome, True)

importing done


In [7]:
# Create a TestSuite
# Balance_suite = unittest.TestLoader().loadTestsFromTestCase(APCWeighingStationHandlerTest)
kuka_suite = unittest.TestLoader().loadTestsFromTestCase(APCKukaRobotHandlerTest)

# Run the tests
unittest.TextTestRunner().run(kuka_suite)


[KMRIIWARobot_1]: (RobotTaskOp - name: test_task - target:Kuka 4              - params: None - location: coordinates:(18, 8) - descriptor: Example Location) is queued
[INFO] [1709210573.594069]: executing task test_task with cmd_seq: 10
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying


.
----------------------------------------------------------------------
Ran 1 test in 19.150s

OK


got the completion msg, moving on


<unittest.runner.TextTestResult run=1 errors=0 failures=0>